# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [ ]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-4886b14e63-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Horror Franchises explorative search") 

Consider the following exploratory scenario:


> we are interested in horror movies, in particular we care about the slasher subgenre, and we want to compare the Halloween and the Friday the 13th francise. 

Background knowledge useful for better conducting the search: Halloween is a movie from 1978 with many sequels and the set of Halloween + all the sequels is called the "Halloween franchise". The same considerations hold for Friday the 13th.


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q221103`     | Halloween       | node |
| `wd:Q1243029`| Friday the 13th       | node |





Also consider

```
wd:Q1243029 ?p ?obj .
```

is the BGP to retrieve all **properties of Friday the 13th**

The workload should


1. Investigate the workers of the two original movies and check any commonality 

2. Check who was the most famous actor/actress on Halloween and Friday the 13th at the time of the release (e.g., the actor/actress who acted in more movies before and/or the actor/actress who won more academy awards)

3. Check which actor/actress did more movies after the release of these two films (consider the films separately)

4. Get all the movies of the two franchises (return year and director)

5. Who is most involved in a franchise (who did something in the movies, e.g., writer, director, producer)? 

6. Investigate which franchise is highly regarded by the critics overall

7. Investigate which franchise costed more 

In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT *
WHERE { 

wd:Q1243029 ?p ?obj .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244'), ('obj', 'https://id.loc.gov/authorities/names/no98033420')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P3212'), ('obj', 'urn:isan:0000-0000-31EA-0000-G-0000-0000-Q')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P4276'), ('obj', 'https://data.cinematheque.qc.ca/data/Work1467')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('obj', 'http://g.co/kg/m/016mng')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('obj', 'friday13th')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('obj', 'm/friday_the_13th_part_1')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('obj', '505')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('obj', 'http://www.wikidata.org/entity/Q1067324')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('obj', 'http://www.wikidata.org/ent

118

# Task 1

*Investigate the workers of the two original movies and check any commonality*

Properties of the film

In [38]:
queryString = """
SELECT distinct ?p ?name
WHERE { 

wd:Q1243029 ?p ?obj .
?p <http://schema.org/name> ?name .
filter (!regex(?name, "ID"))
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('name', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('name', 'performer')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2047'), ('name', 'duration')]
[('p', 'http://www.wikidata.org/prop/direct/P2061'), ('name', 'aspect ratio')]
[('p', 'http://www.wikidata.org/prop/direct/P272'), ('name', 'production comp

36

Propetries that enter the film


In [3]:
queryString = """
SELECT distinct ?p ?name
WHERE { 

?obj ?p wd:Q1243029.
?p <http://schema.org/name> ?name .

} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P6166'), ('name', 'quotes work')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P1441'), ('name', 'present in work')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]


5

In [6]:
# cast, performers, directors, driectors of photography, screenwriters, composers of Halloween
queryString = """
SELECT DISTINCT ?name where {
{
SELECT distinct ?name
WHERE { 
    wd:Q221103 wdt:P175|wdt:P162|wdt:P161|wdt:P334|wdt:P57|wdt:P58|wdt:P86|wdt:P2554|wdt:P750|wdt:P272 ?x .
    
    ?x <http://schema.org/name> ?name .
} 
}
UNION{
SELECT distinct ?name
WHERE { 
    ?x wdt:P161 wd:Q221103 .
    ?x <http://schema.org/name> ?name .
} 
}

}
"""

print("Results")
run_query(queryString)

Results
[('name', 'Arthur Malet')]
[('name', 'Tony Moran')]
[('name', 'Jamie Lee Curtis')]
[('name', 'Nancy Kyes')]
[('name', 'Tommy Lee Wallace')]
[('name', 'Charles Cyphers')]
[('name', 'John Carpenter')]
[('name', 'Debra Hill')]
[('name', 'Nick Castle')]
[('name', 'Netflix')]
[('name', 'Kyle Richards')]
[('name', 'Donald Pleasence')]
[('name', 'Gabriel Cattand')]
[('name', 'Moustapha Akkad')]
[('name', 'P. J. Soles')]
[('name', 'Irwin Yablans')]
[('name', 'Compass International Pictures')]
[('name', 'Nancy Stephens')]
[('name', 'John Michael Graham')]


19

In [7]:
# cast, performers, directors, driectors of photography, screenwriters, composers of Halloween
queryString = """
SELECT DISTINCT ?name where {
{
SELECT distinct ?name
WHERE { 
    wd:Q1243029 wdt:P175|wdt:P162|wdt:P161|wdt:P334|wdt:P57|wdt:P58|wdt:P86|wdt:P2554|wdt:P750|wdt:P272 ?x .
    
    ?x <http://schema.org/name> ?name .
} 
}
UNION{
SELECT distinct ?name
WHERE { 
    ?x wdt:P161 wd:Q1243029 .
    ?x <http://schema.org/name> ?name .
} 
}

}
"""

print("Results")
run_query(queryString)

Results
[('name', 'Virginia Field')]
[('name', 'Harry Manfredini')]
[('name', 'FandangoNow')]
[('name', 'Netflix')]
[('name', 'Paramount Pictures')]
[('name', 'Adrienne King')]
[('name', 'Sean S. Cunningham')]
[('name', 'Betsy Palmer')]
[('name', 'Rex Everhart')]
[('name', 'Robbi Morgan')]
[('name', 'Kevin Bacon')]
[('name', 'Victor Miller')]
[('name', 'Harry Crosby')]
[('name', 'Laurie Bartram')]
[('name', 'Jeannine Taylor')]
[('name', 'Ari Lehman')]
[('name', "What's the Matter with Helen?")]


17

I can just check the things that are connected in anyway to both films

In [8]:
# ANSWER 1)
queryString = """
SELECT ?name ?name2 WHERE {
{
SELECT distinct ?name ?name2
WHERE { 
    wd:Q221103 ?y ?x .
    wd:Q1243029 ?z ?x .
    ?x <http://schema.org/name> ?name .
    ?y <http://schema.org/name> ?name2 .
}}UNION
{SELECT distinct ?name ?name2
WHERE { 
    ?y ?x wd:Q221103.
    ?z ?x wd:Q1243029.
    ?y <http://schema.org/name> ?name .
    ?x <http://schema.org/name> ?name2 .
}}
}
"""
print("Results")
run_query(queryString)

Results
[('name', 'film'), ('name2', 'instance of')]
[('name', 'English'), ('name2', 'original language of film or TV show')]
[('name', 'color'), ('name2', 'color')]
[('name', 'United States of America'), ('name2', 'country of origin')]
[('name', 'Bechdel test'), ('name2', 'assessment')]
[('name', 'video on demand'), ('name2', 'distribution format')]
[('name', 'Netflix'), ('name2', 'distributed by')]
[('name', 'Category IV'), ('name2', 'Filmiroda rating')]
[('name', 'slasher film'), ('name2', 'genre')]
[('name', 'Halloween'), ('name2', 'follows')]
[('name', 'Halloween II'), ('name2', 'follows')]
[('name', 'Scary Movie'), ('name2', 'quotes work')]
[('name', 'Halloween'), ('name2', 'based on')]
[('name', 'Haddonfield'), ('name2', 'present in work')]
[('name', 'Official Halloween Parody'), ('name2', 'based on')]


15

***

# Task 2

*Check who was the most famous actor/actress on Halloween and Friday the 13th at the time of the release (e.g., the actor/actress who acted in more movies before and/or the actor/actress who won more academy awards)*

I think that awards better represent if an artist is known or not.

In [37]:
# Available properties for cast members
queryString = """
SELECT distinct ?name ?x
WHERE { 
  wd:Q1243029 wdt:P161 ?p .
  ?p ?x ?y .
  ?x <http://schema.org/name> ?name .
  filter (!regex(?name, "ID"))
} 
"""

print("Results")
run_query(queryString)

Results
[('name', 'occupation'), ('x', 'http://www.wikidata.org/prop/direct/P106')]
[('name', 'manner of death'), ('x', 'http://www.wikidata.org/prop/direct/P1196')]
[('name', 'filmography'), ('x', 'http://www.wikidata.org/prop/direct/P1283')]
[('name', 'instrument'), ('x', 'http://www.wikidata.org/prop/direct/P1303')]
[('name', 'described by source'), ('x', 'http://www.wikidata.org/prop/direct/P1343')]
[('name', 'genre'), ('x', 'http://www.wikidata.org/prop/direct/P136')]
[('name', 'religion'), ('x', 'http://www.wikidata.org/prop/direct/P140')]
[('name', 'nominated for'), ('x', 'http://www.wikidata.org/prop/direct/P1411')]
[('name', 'languages spoken, written or signed'), ('x', 'http://www.wikidata.org/prop/direct/P1412')]
[('name', 'name in native language'), ('x', 'http://www.wikidata.org/prop/direct/P1559')]
[('name', 'award received'), ('x', 'http://www.wikidata.org/prop/direct/P166')]
[('name', 'image'), ('x', 'http://www.wikidata.org/prop/direct/P18')]
[('name', 'place of birth'

44

In [61]:
# awards of a cast member (properties)
queryString = """
SELECT distinct ?y ?name
WHERE { 
   ?zz wdt:P161 ?p .
  ?p wdt:P166 ?x .
  ?x ?y ?z .
  ?y <http://schema.org/name> ?name .
  filter (?zz=wd:Q1243029 || ?zz=wd:Q221103  )
} 
"""

print("Results")
run_query(queryString)

Results
[('y', 'http://www.wikidata.org/prop/direct/P3729'), ('name', 'next lower rank')]
[('y', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('y', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('y', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('y', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('y', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('y', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('y', 'http://www.wikidata.org/prop/direct/P1705'), ('name', 'native label')]
[('y', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series')]
[('y', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('y', 'http://www.wikidata.org/prop/direct/P180'), ('name', 'depicts')]
[('y', 'http://www.wikidata.org/prop/direct/P1813'), ('name', 'short name')]
[('y', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'differ

35

In [62]:
# Who won awards in general in the cast of both (with dates if possible)
queryString = """

    SELECT ?x ?name ?adate
    WHERE { 
      ?zz wdt:P161 ?p .
      ?p wdt:P166 ?x .
      optional{?x wdt:P585|wdt:P580 ?adate .}
      ?p <http://schema.org/name> ?name .
      filter (?zz=wd:Q1243029 || ?zz=wd:Q221103  )
    } 
         

"""

print("Results")
run_query(queryString)


Results
[('x', 'http://www.wikidata.org/entity/Q105447'), ('name', 'Kevin Bacon'), ('adate', '1972-01-01T00:00:00Z')]
[('x', 'http://www.wikidata.org/entity/Q18600734'), ('name', 'Kevin Bacon'), ('adate', '2015-01-25T00:00:00Z')]
[('x', 'http://www.wikidata.org/entity/Q1011564'), ('name', 'Jamie Lee Curtis')]
[('x', 'http://www.wikidata.org/entity/Q17985761'), ('name', 'Jamie Lee Curtis')]
[('x', 'http://www.wikidata.org/entity/Q787123'), ('name', 'Jamie Lee Curtis')]
[('x', 'http://www.wikidata.org/entity/Q10762848'), ('name', 'Donald Pleasence')]
[('x', 'http://www.wikidata.org/entity/Q251542'), ('name', 'Kevin Bacon')]
[('x', 'http://www.wikidata.org/entity/Q17985761'), ('name', 'Kevin Bacon')]


8

In [67]:
# The only one with dates is Kevin Bacon, and he is only in Friday the 13th
queryString = """
SELECT * where{{
SELECT (max(?pdate) as ?x)
WHERE { 
  wd:Q1243029 wdt:P577 ?pdate .
} 
}{
SELECT ?adate ?name ?name2
WHERE { 
  wd:Q1243029 wdt:P161 ?p .
  ?p wdt:P166 ?x .
  ?x wdt:P585|wdt:P580 ?adate .
  ?p <http://schema.org/name> ?name .
  ?x <http://schema.org/name> ?name2 .
} 
}
               filter(?adate<?x)
}

"""
# ANSWER 2) Kevin Bacon
print("Results")
run_query(queryString)

Results
[('x', '1980-11-10T00:00:00Z'), ('adate', '1972-01-01T00:00:00Z'), ('name', 'Kevin Bacon'), ('name2', 'Saturn Award')]


1

***

# Task 3

*Check which actor/actress did more movies after the release of these two films (consider the films separately)*

In [6]:
# Check for connections from actors to films (there are none)
queryString = """
select distinct ?x ?name
where{
  wd:Q1243029 wdt:P161 ?p .
  ?p ?x ?y . 
  ?x <http://schema.org/name> ?name .
  filter(!regex(?name, "ID"))
  
  }
"""
print("Results")
run_query(queryString)

Results
[('x', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('x', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('x', 'http://www.wikidata.org/prop/direct/P1283'), ('name', 'filmography')]
[('x', 'http://www.wikidata.org/prop/direct/P1303'), ('name', 'instrument')]
[('x', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('x', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('x', 'http://www.wikidata.org/prop/direct/P140'), ('name', 'religion')]
[('x', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('x', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('x', 'http://www.wikidata.org/prop/direct/P1559'), ('name', 'name in native language')]
[('x', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('x', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('x', 'http://www.wikidat

44

In [8]:
# films done by the actors of friday the 13th
queryString = """
select ?x ?p ?mdate
where{
  wd:Q1243029 wdt:P161 ?p .
  ?x wdt:P161 ?p .
  ?x wdt:P577 ?mdate .
  }
"""
print("Results")
run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q757156'), ('p', 'http://www.wikidata.org/entity/Q2440012'), ('mdate', '1979-01-01T00:00:00Z')]
[('x', 'http://www.wikidata.org/entity/Q757156'), ('p', 'http://www.wikidata.org/entity/Q2440012'), ('mdate', '1979-07-12T00:00:00Z')]
[('x', 'http://www.wikidata.org/entity/Q1243009'), ('p', 'http://www.wikidata.org/entity/Q2440012'), ('mdate', '1981-04-30T00:00:00Z')]
[('x', 'http://www.wikidata.org/entity/Q1243009'), ('p', 'http://www.wikidata.org/entity/Q2440012'), ('mdate', '1981-11-02T00:00:00Z')]
[('x', 'http://www.wikidata.org/entity/Q1243029'), ('p', 'http://www.wikidata.org/entity/Q2440012'), ('mdate', '1980-05-09T00:00:00Z')]
[('x', 'http://www.wikidata.org/entity/Q1243029'), ('p', 'http://www.wikidata.org/entity/Q2440012'), ('mdate', '1980-10-23T00:00:00Z')]
[('x', 'http://www.wikidata.org/entity/Q1243029'), ('p', 'http://www.wikidata.org/entity/Q2440012'), ('mdate', '1980-11-10T00:00:00Z')]
[('x', 'http://www.wikidata.org/entity/Q47

220

In [10]:
# ANSWER to 3 part 1
queryString = """
SELECT (count(?x) as ?film) ?person where{
{
  SELECT (max(?pdate) as ?x1)
  WHERE { 
    wd:Q1243029 wdt:P577 ?pdate .
  } 
}
{
  select ?x ?person ?mdate
  where{
    wd:Q1243029 wdt:P161 ?p .
    ?x wdt:P161 ?p .
    ?x wdt:P577 ?mdate .
    ?p <http://schema.org/name> ?person .
  }
}
  filter(?mdate>?x1)             
} group by ?person
order by desc (?film)
"""
print("Results")
run_query(queryString)

Results
[('film', '106'), ('person', 'Kevin Bacon')]
[('film', '19'), ('person', 'Adrienne King')]
[('film', '10'), ('person', 'Betsy Palmer')]
[('film', '7'), ('person', 'Robbi Morgan')]
[('film', '5'), ('person', 'Rex Everhart')]
[('film', '2'), ('person', 'Jeannine Taylor')]
[('film', '2'), ('person', 'Ari Lehman')]


7

In [5]:
# ANSWER to 3 part 2
queryString = """
SELECT (count(?x) as ?film) ?person where{
{
  SELECT (max(?pdate) as ?x1)
  WHERE { 
    wd:Q221103 wdt:P577 ?pdate .
  } 
}
{
  SELECT ?x ?person ?mdate
  WHERE{
    wd:Q221103 wdt:P161 ?p .
    ?x wdt:P161 ?p .
    ?x wdt:P577 ?mdate .
    ?p <http://schema.org/name> ?person .
  }
}
filter(?mdate>?x1)         
} group by ?person
order by desc (?film)
"""
print("Results")
run_query(queryString)

Results
[('film', '105'), ('person', 'Jamie Lee Curtis')]
[('film', '105'), ('person', 'Donald Pleasence')]
[('film', '31'), ('person', 'P. J. Soles')]
[('film', '28'), ('person', 'John Carpenter')]
[('film', '16'), ('person', 'Arthur Malet')]
[('film', '13'), ('person', 'Charles Cyphers')]
[('film', '12'), ('person', 'Nick Castle')]
[('film', '11'), ('person', 'Gabriel Cattand')]
[('film', '8'), ('person', 'Kyle Richards')]
[('film', '5'), ('person', 'Nancy Stephens')]
[('film', '5'), ('person', 'Nancy Kyes')]
[('film', '3'), ('person', 'Tony Moran')]


12

***

# Task 4

*Get all the movies of the two franchises (return year and director)*

In [12]:
# Extract the franchises of the 2 films
queryString = """
SELECT distinct ?x where{
  ?film wdt:P179 ?x
  filter (?film=wd:Q221103 || ?film=wd:Q1243029)
}
"""
print("Results")
run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q1138878')]
[('x', 'http://www.wikidata.org/entity/Q1364022')]


2

In [16]:
# ANSWER to 4
queryString = """
SELECT distinct ?filmn ?film ?director ?date where{
  ?film wdt:P179 ?y .
  ?film wdt:P57 ?director .
  ?film wdt:P577 ?date .

  ?film <http://schema.org/name> ?filmn .
   filter(?y=wd:Q1364022 || ?y=wd:Q1138878)
}
"""
print("Results")
run_query(queryString)

Results
[('filmn', 'Friday the 13th Part 2'), ('film', 'http://www.wikidata.org/entity/Q1243009'), ('director', 'http://www.wikidata.org/entity/Q709325'), ('date', '1981-04-30T00:00:00Z')]
[('filmn', 'Friday the 13th Part 2'), ('film', 'http://www.wikidata.org/entity/Q1243009'), ('director', 'http://www.wikidata.org/entity/Q107404'), ('date', '1981-04-30T00:00:00Z')]
[('filmn', 'Friday the 13th Part 2'), ('film', 'http://www.wikidata.org/entity/Q1243009'), ('director', 'http://www.wikidata.org/entity/Q709325'), ('date', '1981-11-02T00:00:00Z')]
[('filmn', 'Friday the 13th Part 2'), ('film', 'http://www.wikidata.org/entity/Q1243009'), ('director', 'http://www.wikidata.org/entity/Q107404'), ('date', '1981-11-02T00:00:00Z')]
[('filmn', 'Friday the 13th Part III'), ('film', 'http://www.wikidata.org/entity/Q1243013'), ('director', 'http://www.wikidata.org/entity/Q107404'), ('date', '1982-08-13T00:00:00Z')]
[('filmn', 'Friday the 13th Part III'), ('film', 'http://www.wikidata.org/entity/Q124

44

***

# Task 5

*Who is most involved in a franchise (who did something in the movies, e.g., writer, director, producer)?*




In [9]:
# ANSWER to 5
queryString = """
SELECT (count(?x) as ?xx) ?name where{
  ?film wdt:P179 ?y .
  ?film ?x ?z .
   filter(?y=wd:Q1364022 || ?y=wd:Q1138878)
   filter(?x in(wdt:P161, wdt:P162, wdt:P175, wdt:P86, wdt:P58, wdt:P57, wdt:P334, wdt:P2554 ))
   ?z <http://schema.org/name> ?name .
}group by ?name
order by desc (?xx)
limit 15
"""
print("Results")
run_query(queryString)

Results
[('xx', '15'), ('name', 'Harry Manfredini')]
[('xx', '13'), ('name', 'John Carpenter')]
[('xx', '7'), ('name', 'Sean S. Cunningham')]
[('xx', '6'), ('name', 'Debra Hill')]
[('xx', '6'), ('name', 'Steve Miner')]
[('xx', '6'), ('name', 'Rob Zombie')]
[('xx', '5'), ('name', 'Jamie Lee Curtis')]
[('xx', '5'), ('name', 'Donald Pleasence')]
[('xx', '4'), ('name', 'Alan Howarth')]
[('xx', '4'), ('name', 'Tommy Lee Wallace')]
[('xx', '4'), ('name', 'Danielle Harris')]
[('xx', '4'), ('name', 'Kane Hodder')]
[('xx', '4'), ('name', 'Frank Mancuso, Jr.')]
[('xx', '4'), ('name', 'Paul Freeman')]
[('xx', '4'), ('name', 'Moustapha Akkad')]


15

***

# Task 6

*Investigate which franchise is highly regarded by the critics overall*

For the critics opinion we have to again investigate awards and the "ratings".

In [10]:
# Check if some film has been nominated for some award
queryString = """
SELECT ?name1 ?name2 ?name3 where{
  ?film wdt:P179 ?y .
  filter(?y=wd:Q1364022 || ?y=wd:Q1138878)
  ?film wdt:P1411 ?x .
  ?x <http://schema.org/name> ?name1 .
  ?y <http://schema.org/name> ?name2 .
  ?film <http://schema.org/name> ?name3 .
}
"""
print("Results")
run_query(queryString)

Results
[('name1', 'Golden Raspberry Award for Worst Picture'), ('name2', 'Friday the 13th'), ('name3', 'Friday the 13th')]
[('name1', 'Golden Raspberry Award for Worst Supporting Actress'), ('name2', 'Friday the 13th'), ('name3', 'Friday the 13th')]


2

I also checked for the "ratings" that I found in the properties of the films/franchise. In the end "ratings" didn't meant ranking so it wasn't useful. Anyway, the only awards are given to Friday the 13th and are about being a bad film, so is safe to say that critics didn't like it much.

In [15]:
# Check one of the rating available (in the end it wasnt the rating I expected)
queryString = """
SELECT ?name1 ?name2 ?name3 where{
  ?film wdt:P179 ?y .
  filter(?y=wd:Q1364022 || ?y=wd:Q1138878)
  ?film wdt:P2758 ?x .
  ?x <http://schema.org/name> ?name1 .
  ?y <http://schema.org/name> ?name2 .
  ?film <http://schema.org/name> ?name3 .
}order by desc(?x)
"""
# ANSWER to 6. Being nominated for worst picture is enough  
print("Results")
run_query(queryString)

Results
[('name1', 'no minors under sixteen'), ('name2', 'Halloween'), ('name3', 'Halloween')]
[('name1', 'no minors under sixteen'), ('name2', 'Friday the 13th'), ('name3', 'Friday the 13th')]
[('name1', 'no minors under sixteen'), ('name2', 'Friday the 13th'), ('name3', 'Freddy vs. Jason')]
[('name1', 'no minors under twelve'), ('name2', 'Halloween'), ('name3', 'Halloween H20: 20 Years Later')]
[('name1', 'no minors under twelve'), ('name2', 'Halloween'), ('name3', 'Halloween: Resurrection')]
[('name1', 'no minors under twelve'), ('name2', 'Friday the 13th'), ('name3', 'Friday the 13th')]
[('name1', 'no minors under twelve'), ('name2', 'Friday the 13th'), ('name3', 'Friday the 13th Part VII: The New Blood')]
[('name1', 'warning'), ('name2', 'Friday the 13th'), ('name3', 'Friday the 13th')]


8

***

# Task 7

*Investigate which franchise costed more*

In [14]:
# Franchise properties
queryString = """
SELECT distinct ?x ?name{
  ?y ?x ?z .
  filter(?y=wd:Q1364022 || ?y=wd:Q1138878)
  ?x <http://schema.org/name> ?name .
  filter(!regex(?name, "ID"))
}
"""
  
print("Results")
run_query(queryString)

Results
[('x', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('x', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('x', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('x', 'http://www.wikidata.org/prop/direct/P1881'), ('name', 'list of characters')]
[('x', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('x', 'http://www.wikidata.org/prop/direct/P272'), ('name', 'production company')]
[('x', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('x', 'http://www.wikidata.org/prop/direct/P364'), ('name', 'original language of film or TV show')]
[('x', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('x', 'http://www.wikidata.org/prop/direct/P3984'), ('name', 'subreddit')]
[('x', 'http://www.wikidata.org/prop/direct/P495'), ('name', 'country of origin')]
[('x', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('x', 'http://www

16

In [23]:
# Properties in the franchise's films
queryString = """
SELECT distinct ?name ?z where{
  ?film wdt:P179 ?y .
  filter(?y=wd:Q1364022 || ?y=wd:Q1138878)
  ?film ?z ?n .
  ?z <http://schema.org/name> ?name .
  filter(!regex(?name, "ID"))    
}
"""
  
print("Results")
run_query(queryString)

Results
[('name', 'narrator'), ('z', 'http://www.wikidata.org/prop/direct/P2438')]
[('name', 'film editor'), ('z', 'http://www.wikidata.org/prop/direct/P1040')]
[('name', 'movement'), ('z', 'http://www.wikidata.org/prop/direct/P135')]
[('name', 'genre'), ('z', 'http://www.wikidata.org/prop/direct/P136')]
[('name', 'nominated for'), ('z', 'http://www.wikidata.org/prop/direct/P1411')]
[('name', 'takes place in fictional universe'), ('z', 'http://www.wikidata.org/prop/direct/P1434')]
[('name', 'based on'), ('z', 'http://www.wikidata.org/prop/direct/P144')]
[('name', 'title'), ('z', 'http://www.wikidata.org/prop/direct/P1476')]
[('name', 'logo image'), ('z', 'http://www.wikidata.org/prop/direct/P154')]
[('name', 'follows'), ('z', 'http://www.wikidata.org/prop/direct/P155')]
[('name', 'has quality'), ('z', 'http://www.wikidata.org/prop/direct/P1552')]
[('name', 'followed by'), ('z', 'http://www.wikidata.org/prop/direct/P156')]
[('name', 'cast member'), ('z', 'http://www.wikidata.org/prop/di

65

In [31]:
# ANSWER to 7
queryString = """
SELECT (sum(?n) as ?sum) ?name where{
  ?film wdt:P179 ?y .
  filter(?y=wd:Q1364022 || ?y=wd:Q1138878)
  ?film wdt:P2130 ?n .
   ?y <http://schema.org/name> ?name . 
}group by ?name
order by desc (?sum)
"""
  
print("Results")
run_query(queryString)

Results
[('sum', '11900000'), ('name', 'Friday the 13th')]
[('sum', '325000'), ('name', 'Halloween')]


2